# Entrega GPU IV - Shared
### Filipe F. Borba  
### Insper
### Super Computação, Prof. Igor Montagner


## Introdução

O objetivo dessa entrega é comparar diversos algoritmos de multiplicação de matrizes para verificar a diferença das implementações em GPU. Com isso, foram utilizados vetores de tamanho size * size, onde size está em [128, 512, 1024]. Ao utilizar um vetor de tamanho 2048 o programa começava a demorar demais para as soluções mais fracas. A máquina utilizada foi uma p2.xlarge da AWS, que possui uma GPU NVIDIA Tesla K80.

## Entrega

In [5]:
%matplotlib inline
import os
import subprocess

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [6]:
key = "/home/filipefborba/Documents/Keys/IgorNvidia.pem"
host = "ec2-user@ec2-54-80-199-115.compute-1.amazonaws.com" # MUDAR O IP AQUI
working_directory = "/home/ec2-user/borba/supercomp/15-gpu-IV/" # MUDAR A PASTA AQUI

In [7]:
# Realizar o build na máquina da aws.
print(subprocess.call(["ssh", "-i", key, host, "cd", working_directory, "&&", "mkdir", "build"]))
print(subprocess.call(["ssh", "-i", key, host, "cd", working_directory+"build/", "&&", "cmake .."]))
print(subprocess.call(["ssh", "-i", key, host, "cd", working_directory+"build/", "&&", "make", "-j4"]))

0
1
2


In [8]:
# Listar os executaveis
output = subprocess.check_output(["ssh", "-i", key, host, "cd", working_directory+"build/", "&&", "ls"])
output = output.decode("utf-8").splitlines()
files = sorted([x for x in output if (x.startswith("naive") or x.startswith("par") or x.startswith("seq") or x.startswith("tiling"))])
files

[]

In [ ]:
# Devolve o output e o tempo de execucao
def run_test(file, vsize):
    output = subprocess.check_output(["ssh", "-i", key, host, "cd", working_directory, "&&",
                                 f"./{file}{vsize}"])
    output = output.decode("utf-8").splitlines()
     
    print(f"--{file}------{vsize}--")
    print(output[0])
    return [file, vsize, float(output[0][6:])]

In [ ]:
# Tempo de Execução
data = []
for v in vector_sizes[:-1]:
    for f in files:
        data.append(run_test(f[:-3], v))

In [4]:
# Converter us para ms
def fix_time(time):
    if (time.endswith("us")):
        return float(time[:-2])/1000
    elif (time.endswith("ms")):
        return float(time[:-2])
    elif (time.endswith("s")):
        return float(time[:-1])*1000
    else:
        print("time not us, ms or s")
        return 0

In [30]:
# Pegar nome e tempo de execucao
def get_name_and_time(output, name):
    if (name == "tiling"):
        name = "multMat"
    if (name == "naive"):
        name = "MatrixMulKernel"
    for s in output:
        found_name = s.find(name)
        if found_name != -1:
            result = s[found_name:found_name+len(name)]
            found_time = re.search('%(.*)s ', s)
            if (found_time == None):
                pass
            else:
                print(found_time.group(1).split()[0], result)
                return found_time.group(1).split()[0], result

In [33]:
# Tempos de alocacao e copia de memoria, alem de kernel.
def run_nvprof_test(file, input_file):
    output = subprocess.check_output(["ssh", "-i", key, host, "cd", working_directory+"build/", "&&",
                                 "nvprof", f"./{file}"], stderr=subprocess.STDOUT)
    output = output.decode("utf-8").splitlines()
    
    print(f"--{file}------{input_file}--")
    kernel_time, kernel_name = get_name_and_time(output, str(file))
    htod_time, htod_name = get_name_and_time(output, "[CUDA memcpy HtoD]")
    dtoh_time, dtoh_name = get_name_and_time(output, "[CUDA memcpy DtoH]")
    malloc_time, malloc_name = get_name_and_time(output, "cudaMalloc")
    kernel_time = fix_time(kernel_time)
    htod_time = fix_time(htod_time)
    dtoh_time = fix_time(dtoh_time)
    malloc_time = fix_time(malloc_time)
    return [file, input_file,
        kernel_time, kernel_name,
        htod_time, htod_name,
        dtoh_time, dtoh_name,
        malloc_time, malloc_name
           ]

In [ ]:
data_gpu = []
gpu_files = ["naive.cu", "tiling.cu"]
for v in vector_sizes[1:]:
    for f in gpu_files:
        data_gpu.append(run_nvprof_test(f[:-3], v))

# Resultados

Aqui estamos preocupados com a diferença de desempenho, então foram plotados gráficos para verificar a diferença entre as soluções.

In [ ]:
df = pd.DataFrame(data, dtype=np.float64)
df

In [ ]:
df2 = pd.DataFrame(data_gpu, dtype=np.float64)
df2

In [ ]:
groups = df.groupby(0)

fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group[1], group[2], marker='o', linestyle='-', ms=5, label=group[0])
plt.title('Tempos para executáveis diferentes')
plt.ylabel('Tempo (ms)')
plt.xlabel('Tamanho de Entrada')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
groups = df2.groupby(0)

fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group[1], group[2], marker='o', linestyle='-', ms=5, label=group[0])
    
plt.title('Tempos para executáveis diferentes')
plt.ylabel('Tempo (ms)')
plt.xlabel('Tamanho de Entrada')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
for key, item in groups:
    print(groups.get_group(key), "\n\n")

Como podemos verificar nos gráficos acima, o tempo de execução dos algoritmos que utilizam a GPU são muito menores, sendo o algoritmo de Tiling o mais eficiente. 